<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<br><h1>Sample Notebook | Building a Classification Model in Kaggle</h1>
Chase Kusterer<br>
Faculty of Analytics<br>
Hult International Business School<br><br><br>

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<br>
<h2>Purpose of this Script</h2>
This script is designed to demonstrate how to develop a model in Kaggle.
<br><br><br>
<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

Below is a starter code that comes with Jupyter Notebook.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br><br>
You will need to import any additional libraries. It's also a good idea to get rid of the cell above once you're used to Jupyter Notebook in Kaggle.<br>

In [ ]:
# modeling library
import sklearn.linear_model                          # linear modeling in scikit-learn

# other model building tools
from sklearn.model_selection import train_test_split # train-test split
from sklearn.metrics import roc_auc_score            # auc score

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br><br>
Make sure to tell Python the path to your training data. If you have no idea what a path is, look to the left and find and hover over <em>Data&nbsp;>&nbsp;Input&nbsp;>&nbsp;spaceship-titanic</em>. A button should appear allowing you to copy the path to your clipboard.<br>

In [ ]:
# importing the training dataset
path             = "/kaggle/input/spaceship-titanic/"
training_dataset = "train.csv"


# reading in the .csv file with pandas
titanic_train    = pd.read_csv(filepath_or_buffer = path + training_dataset)


# checking basic info about the dataset
titanic_train.info(verbose = True)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
The test set is also available. Notice that there is no data for Transported (y-variable) in the test set. This is intentional.<br>

In [ ]:
# importing the training dataset
path             = "/kaggle/input/spaceship-titanic/"
testing_dataset  = 'test.csv'

# importing the testing dataset
titanic_test = pd.read_csv(filepath_or_buffer = path + testing_dataset)

# checking basic info about the dataset
titanic_test.info(verbose = True)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
You may want to join the datasets together before feature engineering. This will help make things more efficient.<br>

In [ ]:
titanic_train['set'] = 'Training'
titanic_test ['set'] = 'Testing'

# concatenating both datasets together for mv and feature engineering
titanic_df = titanic_train.append(other = titanic_test)

# resetting index to avoid problems later in the code
titanic_df.reset_index(drop = False,
                       inplace = True)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
Let's look at correlations to find an x-variable to model with.<br>

In [ ]:
# instantiating a correlation matrix
titanic_corr = titanic_train.corr(method = 'pearson').round(decimals = 2)

# transforming correlations to absolute values
titanic_corr.loc[ : , 'Transported' ].apply(func = abs).sort_values(ascending = False)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
Since training and testing sets have already been developed for us, we can simply select the X-features we want to model with and we're good to go.<br>

In [ ]:
# imputing in missing values for RoomService
titanic_df[ 'RoomService' ].fillna(value = 0, inplace = True)

# setting explanatory variable(s) with most correlated x-variable
x_train = titanic_df[ ['RoomService'] ][ titanic_df['set'] == 'Training' ]

# setting response variable
y_train = titanic_df[ 'Transported' ][ titanic_df['set']   == 'Training' ]

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
Even though there's already a testing set, it's a good idea to build one on the training data to assess model performance and stability qualities.<br>

In [ ]:
# developing training and validation sets
x_train_1, x_train_2, y_train_1, y_train_2 = train_test_split(
            x_train,
            y_train.astype(dtype = 'int'),
            random_state = 123,
            test_size    = 0.25,
            stratify     = y_train)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
Below is a Logistic Regression model using scikit-learn. Note that as you explore additional scikit-learn models, the only things that need to change are the model name and the model type.<br>

In [ ]:
# picking a model name
model_name = "Logistic Regression"


# INSTANTIATING a model object - CHANGE THIS AS NEEDED
model = sklearn.linear_model.LogisticRegression()


# FITTING to the training data
model_fit = model.fit(x_train_1, y_train_1)


# PREDICTING on the response variable
model_train_pred = model_fit.predict(x_train_1)
model_valid_pred = model_fit.predict(x_train_2)


# SCORING the results (accuracy)
model_train_score = model.score(x_train_1, y_train_1).round(4) # training accuracy
model_valid_score = model.score(x_train_2, y_train_2).round(4) # validation accuracy

# SCORING the results (auc)
model_train_auc = roc_auc_score(y_true  = y_train_1,
                                y_score = model_train_pred).round(decimals = 4)

model_valid_auc = roc_auc_score(y_true  = y_train_2,
                                y_score = model_valid_pred).round(decimals = 4)

# displaying results
print('Training Accuracy:  ', model_train_score)
print('Validation Accuracy:', model_valid_score)
print('Training AUC:       ', model_train_auc)
print('Validation AUC:     ', model_valid_auc)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h2>Creating and Submitting Predictions</h2>
In this competition, your results should be submitted as a .csv file. This file should contain exactly two columns:<br><br>

1. Id (from the test set)
2. SalePrice (predictions from your model)

<br>
The following code with do exactly that. Notice that all we need to do is apply the same X-variables (x_data) to the .predict step to generate our predictions.

In [ ]:
# setting x_test
x_test  = titanic_df[ ['RoomService'] ][ titanic_df['set'] == 'Testing' ]

# PREDICTING on new data
model_pred = model.predict(x_test)

# checking results
model_pred

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
From here, we simply create a DataFrame with the original passenger ids (&nbsp;<em>PassengerId</em>&nbsp;) and the predicted values from the model (&nbsp;<em>Transported</em>&nbsp;).<br>

In [ ]:
# saving predictions with their respective Ids from the test set
predictions = pd.DataFrame(data = { 'PassengerId' : titanic_test['PassengerId'],
                                    'Transported' : model_pred.astype(bool)               } )

# checking the results
predictions.head(n = 5)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
Finally, we save the DataFrame as a .csv file and we're ready to submit. This file will be available in the <em>/kaggle/working</em> directory.

In [ ]:
# sending predictions to .csv file
predictions.to_csv(path_or_buf = 'submission.csv',
                   index = False)

<br>

~~~

  _    _                            _____          _ _             _ 
 | |  | |                          / ____|        | (_)           | |
 | |__| | __ _ _ __  _ __  _   _  | |     ___   __| |_ _ __   __ _| |
 |  __  |/ _` | '_ \| '_ \| | | | | |    / _ \ / _` | | '_ \ / _` | |
 | |  | | (_| | |_) | |_) | |_| | | |___| (_) | (_| | | | | | (_| |_|
 |_|  |_|\__,_| .__/| .__/ \__, |  \_____\___/ \__,_|_|_| |_|\__, (_)
              | |   | |     __/ |                             __/ |  
              |_|   |_|    |___/                             |___/   

~~~


<br>